In [16]:
import cv2

In [17]:
import matplotlib.pyplot as plt

In [18]:
import tensorflow as tf
print(tf.__version__)
%matplotlib inline

2.0.0


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [21]:
image_gen=ImageDataGenerator(rotation_range=30,#rotate the image 30 degrees
                             width_shift_range=0.1,#shift the pic width by amax of 10%
                             height_shift_range=0.1,#shift the pic height by amax of 10%
                             rescale=1/255,#rescale tghe image by normalizing it
                             shear_range=0.2,#shear means cutting away part of the image(max 20%)
                             zoom_range=0.2,# Zoom in by 20% max
                             horizontal_flip=True,#allow horizontal flipping
                             fill_mode='nearest',#fill in  missing pixel with the nearest filled value
                             validation_split=0.2
                            )

In [22]:
# width,height,channel
image_shape=(150,150,3)


Creating the Model

In [23]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(150,150,3),activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(150,150,3),
                activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),input_shape=(150,150,3),
                activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),input_shape=(150,150,3),
                activation='relu',))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))


model.add(Dense(512))
model.add(Activation('relu'))


#DropOuts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of the Neurons


# Last layer for 0=paper and 1=rock and 2=scissors
model.add(Dense(3))
model.add(Activation('softmax'))



model.compile(loss='categorical_crossentropy',#Computes the cross-entropy loss between true labels and predicted labels.
             optimizer='rmsprop',#The RMSprop optimizer is similar to the gradient descent algorithm with momentum. The RMSprop optimizer restricts the oscillations in the vertical direction. Therefore, we can increase our learning rate and our algorithm could take larger steps in the horizontal direction converging faster
             metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

In [25]:


train_image_gen=image_gen.flow_from_directory(r'C:\Users\Harshil\Downloads\rps',target_size=image_shape[:2],
                                             class_mode='categorical',
                                             subset='training')



Found 2016 images belonging to 3 classes.


In [26]:

validation_image_gen=image_gen.flow_from_directory(r'C:\Users\Harshil\Downloads\rps',target_size=image_shape[:2],
                                             class_mode='categorical',
                                             subset='validation')

Found 504 images belonging to 3 classes.


In [27]:
test_image_gen=image_gen.flow_from_directory(r'C:\Users\Harshil\Downloads\rps-test-set',target_size=image_shape[:2],
                                             class_mode='categorical')

Found 372 images belonging to 3 classes.


In [28]:
train_image_gen.class_indices

{'paper': 0, 'rock': 1, 'scissors': 2}

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
results=model.fit_generator(train_image_gen,epochs=15,
                           validation_data=validation_image_gen,
                            validation_steps=8)

Epoch 1/15
63/63 [==============================] - 271s 4s/step - loss: 1.1679 - accuracy: 0.5149 - val_loss: 0.8713 - val_accuracy: 0.4922
Epoch 2/15
63/63 [==============================] - 164s 3s/step - loss: 0.6684 - accuracy: 0.7460 - val_loss: 0.5383 - val_accuracy: 0.7617
Epoch 3/15
63/63 [==============================] - 162s 3s/step - loss: 0.3835 - accuracy: 0.8472 - val_loss: 0.5495 - val_accuracy: 0.6992
Epoch 4/15
63/63 [==============================] - 170s 3s/step - loss: 0.2462 - accuracy: 0.9082 - val_loss: 0.5021 - val_accuracy: 0.7930
Epoch 5/15
63/63 [==============================] - 162s 3s/step - loss: 0.1708 - accuracy: 0.9449 - val_loss: 0.1914 - val_accuracy: 0.9336
Epoch 6/15
63/63 [==============================] - 165s 3s/step - loss: 0.1265 - accuracy: 0.9588 - val_loss: 1.1483 - val_accuracy: 0.6445
Epoch 7/15
63/63 [==============================] - 164s 3s/step - loss: 0.1099 - accuracy: 0.9583 - val_loss: 0.5453 - val_accuracy: 0.8398
Epoch 8/15
63

In [31]:
model.save('rps3.h5')

In [41]:
loss, accuracy = model.evaluate(test_image_gen)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

12/12 [==============================] - 10s 804ms/step - loss: 0.1083 - accuracy: 0.9651
Loss:  0.1082633084927996
Accuracy:  0.96505374


In [47]:
results_2 = model.evaluate(test_image_gen, verbose=2)


12/12 - 9s - loss: 0.0773 - accuracy: 0.9624


In [ ]:
plt.plot(results.history['accuracy'])